In this notebook we will see how to create safetensors 

We have to keep the following things in mind 
- [ ] one csv and one pt files
- [ ] can have multiple csv files and multiple pt files
- [ ] preprocessing before saving as safe tensor
- [ ] we will save only bboxes once. - there is an idea to store multiple different bounding boxes for the same image but we have to handle this at all transforms. keeping this as TODO for now 

In [70]:
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp create_ds

In [1]:
#| export 
import torch
import pandas as pd
import numpy as np 
import fastcore.all as fc 

from loguru import logger
from typing import Optional
from safetensors.numpy import save_file
from qct_utils.ctscan.ct_loader import CTAnnotLoader
from qct_utils.apps.nodule_diameter import get_major_minor_coordinates_3d_nodule

In [2]:
from qct_utils.schema.ct import ChestCTMaster

In [3]:
from qct_utils.ctvis.viewer import plot_scans
from qct_utils.ctvis.vis_cct import plot_boxes_ctscan
from qct_utils.ctvis.draw import draw_bboxes_and_titles
from qct_data.series_meta import get_qct_collection

In [4]:
df_det = pd.read_csv("/home/users/shubham.kumar/projects/qct_data/qct_data_status/det.csv")

In [5]:
datasets = df_det.dataset.unique().tolist()

In [71]:
dataset = 'recist_segmed'

In [7]:
dataset

'wcg'

In [8]:
datasets

['dedomena',
 'dsb',
 'internal',
 'lidc',
 'medframe',
 'mgh',
 'nlst',
 'qxr_fda',
 'segmed_pm',
 'wcg']

In [72]:
meta_root = get_qct_collection("recist_segmed")

In [73]:
meta_root.count()

1926

In [74]:
# meta_root = f"/cache/fast_data_nas72/qct/data_governance/series_dicts/wcg_fda_qxr.pt"


In [75]:
import os

In [76]:
# # meta_root = "/cache/fast_data_nas72/qct/data_governance/series_dicts/dsb.pt"
# csv_loc = f"/home/users/shubham.kumar/projects/qct_data/qct_data_status/det/data/{dataset}_merged.csv"
# save_dir = f"/cache/fast_data_nas8/qct/shubham/cache_3d/{dataset}/"


In [77]:
# Specify the directory path you want to create
path = fc.Path(f"/cache/fast_data_nas8/qct/shubham/cache_3d/{dataset}/")
path.mkdir(parents=True, exist_ok=True)

lungmask_path = fc.Path(f"/cache/fast_data_nas8/qct/shubham/lung_mask_cache_det/{dataset}/")
lungmask_path.mkdir(parents=True, exist_ok=True)


In [15]:
csv_loc = f"/home/users/shubham.kumar/projects/interns/qct_data/qct_data_status/{dataset}/data/merged.csv"
df1 = pd.read_csv(csv_loc)
df1

,scan_name,annot_union,meta_union,rads
0,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (130, 261, 62, 6...",{'internalstructure': 'internalStructure_soft_...,4
1,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (90, 236, 100, 1...",{'internalstructure': 'internalStructure_soft_...,2
2,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (135, 213, 99, 3...",{'internalstructure': 'internalStructure_soft_...,2
3,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (95, 120, 229, 4...","{'calcification': 'no', 'spiculation': 'no', '...",1
4,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (59, 331, 247, 1...",{'internalstructure': 'internalStructure_soft_...,2
...,...,...,...,...
2484,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (194, 265, 15, 5...","{'calcification': 'no', 'spiculation': 'no', '...",1
2485,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (150, 312, 29, 1...","{'calcification': 'no', 'spiculation': 'no', '...",1
2486,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (314, 285, 58, 1...","{'calcification': 'no', 'spiculation': 'no', '...",1
2487,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (293, 280, 60, 5...","{'calcification': 'no', 'spiculation': 'no', '...",1


In [16]:
csv_loc = f"/home/users/shubham.kumar/projects/qct_data/qct_data_status/{dataset}/data/merged.csv"
df2 = pd.read_csv(csv_loc)
df2

,scan_name,annot_union,meta_union,rads
0,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (130, 261, 62, 6...",{'internalstructure': 'internalStructure_soft_...,4
1,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (90, 236, 100, 1...",{'internalstructure': 'internalStructure_soft_...,2
2,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (135, 213, 99, 3...",{'internalstructure': 'internalStructure_soft_...,2
3,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (95, 120, 229, 4...","{'calcification': 'no', 'spiculation': 'no', '...",1
4,1.3.6.1.4.1.55648.1123560832525979781356990547...,"{'mask': {'rle': {'encoding': (59, 331, 247, 1...",{'internalstructure': 'internalStructure_soft_...,2
...,...,...,...,...
2527,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (194, 265, 15, 5...","{'calcification': 'no', 'spiculation': 'no', '...",1
2528,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (150, 312, 29, 1...","{'calcification': 'no', 'spiculation': 'no', '...",1
2529,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (314, 285, 58, 1...","{'calcification': 'no', 'spiculation': 'no', '...",1
2530,1.3.6.1.4.1.55648.1559945288373937017347087763...,"{'mask': {'rle': {'encoding': (293, 280, 60, 5...","{'calcification': 'no', 'spiculation': 'no', '...",1


In [17]:
df = pd.concat([df1,df2])

In [18]:
df = df[df.duplicated(keep=False) == False]


In [19]:
df = df.rename(columns = {"annot_union" : "annot" , "meta_union": "meta"})

In [20]:
import pandas as pd

df = df.sort_values(by='scan_name')
df['counter'] = df.groupby('scan_name').cumcount() + 1
df['annot_id'] = df['scan_name'] + "_" + df['counter'].astype(str)
df.drop(columns=['counter'], inplace=True)


In [21]:
df.to_csv("a.csv" , index = False)

In [22]:
# df["meta"] = df["meta"].apply(lambda x : eval(x))

In [78]:
csv_loc = "/home/users/shubham.kumar/projects/recist/recist_annotation_batch1_6.csv"
df = pd.read_csv(csv_loc)
sids = df.annot_id.values
series_ids = np.unique([sid.rsplit("_")[0]  for sid in sids])

In [23]:
df.meta.values[0]

"{'calcification': 'no|no', 'spiculation': 'no|no', 'texture': 'part_solid|ground_glass_nodule', 'location': 'LUL|LUL', 'most_clinically_significant_nodule': True, 'short_axis_diameter': '12|7', 'long_axis_diameter': '17|14'}"

In [24]:
series_ids = fc.L(df.scan_name.unique().tolist())

In [25]:
len(series_ids)

22

In [26]:
series_ids

(#22) ['1.3.6.1.4.1.55648.100188019674846432076644931943449358309.3','1.3.6.1.4.1.55648.10826984285070099374502931340697219727.2','1.3.6.1.4.1.55648.112019282314479012696535730268202930787.4','1.3.6.1.4.1.55648.142871668293212255326909887709318800673.3','1.3.6.1.4.1.55648.14472369168369316704390341315926470755.3','1.3.6.1.4.1.55648.145225188666868454776927611389998620353.4','1.3.6.1.4.1.55648.153217532010059645291405450535823896249.2','1.3.6.1.4.1.55648.184549866740890328498751120900044094933.5','1.3.6.1.4.1.55648.2.219368117248306245022073007232919993556','1.3.6.1.4.1.55648.2.276038934349685497643197766989461163236'...]

## we need to see what all files we need to create the safetensors for?

In [79]:
ds = CTAnnotLoader(scans_root=meta_root, csv_loc=csv_loc, series_ids=series_ids,dummy_scans= False)

In [80]:
len(ds)

136

In [66]:
from qct_utils.schema.ct.chestct import RecistNode


In [67]:
RecistNode

qct_utils.schema.ct.chestct.RecistNode

In [69]:
sid = series_ids[np.random.randint(len(series_ids))]
ctscan = ds[sid]

In [63]:
ctscan

ChestCTMaster(study_info=None, series_info=None, scan=CTScan(array=array([[[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048]],

       [[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048]],

       [[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., 

In [29]:
kk = plot_boxes_ctscan(ctscan)

AttributeError: 'NoneType' object has no attribute 'xcyczcwhd'

In [ ]:
plot_scans([kk, ], ["scan"])

> Everything seems okay. lets process this and stip of the extra un-necessary stuff

In [ ]:
file = {}
spacing = ctscan.scan.spacing
file["series_id"] = ctscan.series_instance_uid
file["images"] = ctscan.scan.array.astype(np.int16)
file["boxes"] =  np.empty((0, 6)) if len(ctscan.nodules) == 0 else np.asarray([i.gt.annot.bbox.xcyczcwhd for i in  ctscan.nodules])[:, [1, 0, 2, 4, 3, 5]]
if np.any(file["boxes"]<0): breakpoint()
file["labels"] = np.zeros(file["boxes"].shape[0]).astype(np.int8)
file["spacing"] = np.asarray([spacing.z, spacing.y, spacing.x])


In [82]:
#| export 
def ctscan2dict(ctscan):
    file = {}
    spacing = ctscan.scan.spacing
    file["series_id"] = ctscan.series_instance_uid
    file["images"] = ctscan.scan.array.astype(np.int16)
    file["boxes"] =  np.empty((0, 6)) if len(ctscan.nodules) == 0 else np.asarray([i.gt.annot.bbox.xcyczcwhd for i in  ctscan.nodules])[:, [1, 0, 2, 4, 3, 5]]
    if np.any(file["boxes"]<0): breakpoint()
    file["labels"] = np.zeros(file["boxes"].shape[0]).astype(np.int8)
    file["spacing"] = np.asarray([spacing.z, spacing.y, spacing.x])
    return file

In [ ]:
file = ctscan2dict(ctscan)

In [83]:
#| export 
def fixed_cache_transforms(lung_cache_dir: str = f"/cache/fast_data_nas8/qct/shubham/lung_mask_cache_det/{dataset}/", 
                           device: str="cuda:1",
                            roi: Optional[str] = None):
    from voxdet.utils import locate_cls
    from torchvision.transforms import Compose
    test_transforms = [
    dict(__class_fullname__ = "voxdet.tfsm.standard.StandardT",
         src_mode="yxzhwd", 
         img_src_mode="zyx"),
    dict(__class_fullname__="voxdet.tfsm.med.AddLungCache", 
         cache_dir=lung_cache_dir,
        device = device), 
    dict(__class_fullname__="voxdet.tfsm.med.CropLung",
         device = device,
        roi = roi),
    ]
    transforms = Compose([locate_cls(i) for i in test_transforms])
    return transforms 

In [ ]:
transforms = fixed_cache_transforms()

In [193]:
transforms

NameError: name 'transforms' is not defined

In [31]:
#| export 
def generate_cache(meta_root, csv_loc, series_ids, transforms, save_dir_root):
    tfsm = transforms
    ds = CTAnnotLoader(scans_root=meta_root, csv_loc=csv_loc, series_ids=series_ids)
    fc.Path(save_dir_root).mkdir(exist_ok=True)
    for n, sid in enumerate(series_ids):
        save_path = fc.Path(save_dir_root)/f"{sid}.safetensors"
        if save_path.exists(): 
            logger.info(f"{n}-{sid} already exists")
            continue
        ctscan = ds[sid]
        for annot in ctscan.nodules:
            annot.gt.annot.bbox = annot.gt.annot.mask.to_boxc()
            
            logger.debug(annot.gt.annot.bbox)
        
        file = ctscan2dict(ctscan)
        file2 = tfsm(file)
        
        if np.any(file2["boxes"]<0): 
            logger.info(f"failed: {sid}")
            continue
        #file["boxes"].shape == 0:
        logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")
        
        file2["images"]=file2["images"].astype(np.int16)
        del file2["lung_mask"] #delete lung mask it is not required.
        del file2["series_id"] # delete series_id as we are saving the file with series_id.
        
        save_file(file2, save_path)

In [84]:
save_dir_root = f"/cache/fast_data_nas8/qct/shubham/cache_3d/{dataset}/"

In [86]:
dataset

'recist_segmed'

In [85]:
tfsm = fixed_cache_transforms()
# tfsm_left = fixed_cache_transforms(roi="left")
# tfsm_right = fixed_cache_transforms(roi = "right")

In [87]:
len(series_ids)

136

In [88]:
series_ids

array(['1.3.6.1.4.1.55648.2.101241734547586775452322831296468201972',
       '1.3.6.1.4.1.55648.2.108093775649662319973295460215823794828',
       '1.3.6.1.4.1.55648.2.117920632119097483675993131555963786967',
       '1.3.6.1.4.1.55648.2.128849357129041971604282259998126150107',
       '1.3.6.1.4.1.55648.2.128899056479476615197431186292328764741',
       '1.3.6.1.4.1.55648.2.134994477946138374466181765108799468380',
       '1.3.6.1.4.1.55648.2.139785551441104934563923221820219450953',
       '1.3.6.1.4.1.55648.2.145415111583706588050030447094741143985',
       '1.3.6.1.4.1.55648.2.154359245579735257644154230598688986318',
       '1.3.6.1.4.1.55648.2.155017639880451634701430179153013295880',
       '1.3.6.1.4.1.55648.2.15891357133363139502089312930887236181',
       '1.3.6.1.4.1.55648.2.169051242249054840910091957539366032497',
       '1.3.6.1.4.1.55648.2.173465986389596934332992029676457345625',
       '1.3.6.1.4.1.55648.2.199074783588045101095971497009454260965',
       '1.3.6.1.4.1.5

In [89]:
from tqdm.auto import tqdm

In [91]:
# ds = CTAnnotLoader(scans_root=meta_root, csv_loc=csv_loc, series_ids=series_ids)
fc.Path(save_dir_root).mkdir(exist_ok=True)
for n, sid in tqdm(enumerate(series_ids)):
    save_path = fc.Path(save_dir_root)/f"{sid}.safetensors"
    if save_path.exists(): 
        logger.info(f"{n}-{sid} already exists")
        continue
#     save_path_left = fc.Path(save_dir_root)/f"{sid}_left.safetensors"
#     save_path_right = fc.Path(save_dir_root)/f"{sid}_right.safetensors"
    
    try : 
        ctscan = ds[sid]
    except:
        logger.debug(f"scan is not present for this {sid}")
        continue
    for annot in ctscan.nodules:
        annot.gt.annot.bbox = annot.gt.annot.mask.to_boxc()

        logger.debug(annot.gt.annot.bbox)

    file = ctscan2dict(ctscan)
    
    rads = []
#     for att in ctscan.nodules : 
#         rads.append(df.loc[df["annot_id"] == att.gt.annot_id , "rads"].values[0])
        
#     file["rads"] = np.asarray(rads)
        
    try:
    
        #################
        file2 = tfsm(file)

        if np.any(file2["boxes"]<0): 
            logger.info(f"failed: {series_id}")
            continue
        #file["boxes"].shape == 0:
        logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")

        file2["images"]=file2["images"].astype(np.int16)
        del file2["lung_mask"] #delete lung mask it is not required.
        del file2["series_id"] # delete series_id as we are saving the file with series_id.

        save_file(file2, save_path)

        #####################
#         file3 = tfsm_left(file)

#         if np.any(file3["boxes"]<0): 
#             logger.info(f"failed: {series_id}")
#             continue
#         #file["boxes"].shape == 0:
#         logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")

#         file3["images"]=file3["images"].astype(np.int16)
#         del file3["lung_mask"] #delete lung mask it is not required.
#         del file3["series_id"] # delete series_id as we are saving the file with series_id.

#         save_file(file3, save_path_left)

        ######################
#         file4 = tfsm_right(file)

#         if np.any(file4["boxes"]<0): 
#             logger.info(f"failed: {series_id}")
#             continue
#         #file["boxes"].shape == 0:
#         logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")

#         file4["images"]=file4["images"].astype(np.int16)
#         del file4["lung_mask"] #delete lung mask it is not required.
#         del file4["series_id"] # delete series_id as we are saving the file with series_id.

#         save_file(file4, save_path_right)
    except Exception as e :
        print(e)
        continue
    except Exception as KeyboardInterrupt :
        break
        
    

0it [00:00, ?it/s]

2024-06-06 17:59:05.686 | DEBUG    | __main__:<module>:19 - xc=151.0 yc=229.0 w=25.0 h=21.0 zc=109.5 d=2.0
2024-06-06 17:59:05.780 | DEBUG    | __main__:<module>:19 - xc=276.0 yc=262.5 w=147.0 h=144.0 zc=93.0 d=3.0
2024-06-06 17:59:05.879 | DEBUG    | __main__:<module>:19 - xc=380.0 yc=187.5 w=11.0 h=12.0 zc=109.5 d=2.0
2024-06-06 17:59:07.892 | INFO     | __main__:<module>:38 - 0-1.3.6.1.4.1.55648.2.101241734547586775452322831296468201972-3
2024-06-06 17:59:11.550 | DEBUG    | __main__:<module>:19 - xc=139.5 yc=217.0 w=42.0 h=45.0 zc=48.0 d=7.0
2024-06-06 17:59:11.619 | DEBUG    | __main__:<module>:19 - xc=210.5 yc=156.0 w=38.0 h=55.0 zc=46.0 d=11.0
2024-06-06 17:59:12.503 | INFO     | __main__:<module>:38 - 1-1.3.6.1.4.1.55648.2.108093775649662319973295460215823794828-2
2024-06-06 17:59:18.510 | DEBUG    | __main__:<module>:19 - xc=319.5 yc=200.0 w=38.0 h=47.0 zc=93.0 d=11.0
2024-06-06 17:59:19.445 | INFO     | __main__:<module>:38 - 2-1.3.6.1.4.1.55648.2.1179206321190974836759931315

2024-06-06 18:00:54.354 | DEBUG    | __main__:<module>:19 - xc=129.5 yc=344.5 w=64.0 h=46.0 zc=57.0 d=17.0
2024-06-06 18:00:54.472 | DEBUG    | __main__:<module>:19 - xc=425.5 yc=234.5 w=30.0 h=30.0 zc=108.0 d=5.0
2024-06-06 18:00:55.408 | INFO     | __main__:<module>:38 - 16-1.3.6.1.4.1.55648.2.210356919545821483516440612856462138874-4
2024-06-06 18:01:01.711 | DEBUG    | __main__:<module>:19 - xc=199.0 yc=184.0 w=43.0 h=107.0 zc=71.0 d=19.0
2024-06-06 18:01:02.798 | INFO     | __main__:<module>:38 - 17-1.3.6.1.4.1.55648.2.222877459935596528555540825320371598683-1
2024-06-06 18:01:06.783 | DEBUG    | __main__:<module>:19 - xc=339.5 yc=243.5 w=10.0 h=12.0 zc=57.0 d=1.0
2024-06-06 18:01:07.443 | INFO     | __main__:<module>:38 - 18-1.3.6.1.4.1.55648.2.224351568696488523843304036205693471780-1
2024-06-06 18:01:12.368 | DEBUG    | __main__:<module>:19 - xc=325.0 yc=237.5 w=91.0 h=94.0 zc=116.5 d=12.0
2024-06-06 18:01:13.529 | INFO     | __main__:<module>:38 - 19-1.3.6.1.4.1.55648.2.229337

2024-06-06 18:02:16.798 | DEBUG    | __main__:<module>:19 - xc=331.0 yc=235.5 w=9.0 h=12.0 zc=100.5 d=2.0
2024-06-06 18:02:18.113 | INFO     | __main__:<module>:38 - 27-1.3.6.1.4.1.55648.2.266278801926372327331017888252595908010-3
2024-06-06 18:02:21.642 | DEBUG    | __main__:<module>:19 - xc=163.5 yc=293.0 w=232.0 h=155.0 zc=40.0 d=65.0
2024-06-06 18:02:23.930 | INFO     | __main__:<module>:38 - 28-1.3.6.1.4.1.55648.2.291577612293947930523529467264106951817-1
2024-06-06 18:02:27.545 | DEBUG    | __main__:<module>:19 - xc=117.5 yc=179.0 w=32.0 h=25.0 zc=33.5 d=12.0
2024-06-06 18:02:27.631 | DEBUG    | __main__:<module>:19 - xc=326.5 yc=374.0 w=8.0 h=9.0 zc=19.5 d=2.0
2024-06-06 18:02:27.714 | DEBUG    | __main__:<module>:19 - xc=311.0 yc=335.5 w=219.0 h=22.0 zc=22.0 d=1.0
2024-06-06 18:02:27.797 | DEBUG    | __main__:<module>:19 - xc=149.5 yc=367.5 w=14.0 h=6.0 zc=22.0 d=1.0
2024-06-06 18:02:28.614 | INFO     | __main__:<module>:38 - 29-1.3.6.1.4.1.55648.2.29844995175615404801669564102

2024-06-06 18:03:55.533 | DEBUG    | __main__:<module>:19 - xc=401.5 yc=280.5 w=66.0 h=76.0 zc=36.0 d=9.0
2024-06-06 18:03:56.461 | INFO     | __main__:<module>:38 - 41-1.3.6.1.4.1.55648.2.338585781052707906137196772154985930528-1
2024-06-06 18:04:02.842 | DEBUG    | __main__:<module>:19 - xc=178.5 yc=235.5 w=72.0 h=62.0 zc=53.5 d=16.0
2024-06-06 18:04:03.679 | INFO     | __main__:<module>:38 - 42-1.3.6.1.4.1.55648.2.362212079136922376613871919623401530515-1
2024-06-06 18:04:07.405 | DEBUG    | __main__:<module>:19 - xc=383.0 yc=202.0 w=129.0 h=153.0 zc=61.0 d=25.0
2024-06-06 18:04:08.616 | INFO     | __main__:<module>:38 - 43-1.3.6.1.4.1.55648.2.377569702146349227046428196539719694413-1
2024-06-06 18:04:12.398 | DEBUG    | __main__:<module>:19 - xc=345.0 yc=263.0 w=29.0 h=63.0 zc=60.0 d=3.0
2024-06-06 18:04:13.001 | INFO     | __main__:<module>:38 - 44-1.3.6.1.4.1.55648.2.37875457283651965956213552964134139891-1
2024-06-06 18:04:18.746 | DEBUG    | __main__:<module>:19 - xc=175.5 yc=2

...

2024-06-06 18:07:51.908 | DEBUG    | __main__:<module>:19 - xc=215.0 yc=298.0 w=17.0 h=15.0 zc=88.5 d=4.0
2024-06-06 18:07:52.018 | DEBUG    | __main__:<module>:19 - xc=206.0 yc=270.0 w=39.0 h=47.0 zc=66.5 d=12.0
2024-06-06 18:07:53.440 | INFO     | __main__:<module>:38 - 78-1.3.6.1.4.1.55648.2.663418360359430961094732627351759687443-4
2024-06-06 18:07:59.565 | DEBUG    | __main__:<module>:19 - xc=333.0 yc=317.5 w=41.0 h=44.0 zc=100.5 d=10.0
2024-06-06 18:08:00.561 | INFO     | __main__:<module>:38 - 79-1.3.6.1.4.1.55648.2.667827628402325551525673901260803687741-1
2024-06-06 18:08:07.032 | DEBUG    | __main__:<module>:19 - xc=224.5 yc=248.5 w=62.0 h=74.0 zc=111.0 d=17.0
2024-06-06 18:08:07.149 | DEBUG    | __main__:<module>:19 - xc=178.5 yc=195.0 w=6.0 h=5.0 zc=114.0 d=1.0
2024-06-06 18:08:07.264 | DEBUG    | __main__:<module>:19 - xc=196.0 yc=153.0 w=5.0 h=5.0 zc=107.0 d=1.0
2024-06-06 18:08:07.381 | DEBUG    | __main__:<module>:19 - xc=190.0 yc=248.0 w=7.0 h=5.0 zc=123.0 d=1.0
2024-0

2024-06-06 18:09:22.908 | DEBUG    | __main__:<module>:19 - xc=331.0 yc=270.5 w=25.0 h=32.0 zc=70.5 d=4.0
2024-06-06 18:09:23.732 | INFO     | __main__:<module>:38 - 90-1.3.6.1.4.1.55648.2.723332597027534921849014487570161809524-1
2024-06-06 18:09:29.890 | DEBUG    | __main__:<module>:19 - xc=375.0 yc=322.5 w=71.0 h=48.0 zc=61.0 d=11.0
2024-06-06 18:09:31.015 | INFO     | __main__:<module>:38 - 91-1.3.6.1.4.1.55648.2.725519559665297291964983311322637833990-1
2024-06-06 18:09:36.338 | DEBUG    | __main__:<module>:19 - xc=385.0 yc=310.0 w=51.0 h=39.0 zc=45.0 d=9.0
2024-06-06 18:09:37.521 | INFO     | __main__:<module>:38 - 92-1.3.6.1.4.1.55648.2.733916660342015658251306918943654766760-1
2024-06-06 18:09:43.667 | DEBUG    | __main__:<module>:19 - xc=347.5 yc=253.0 w=126.0 h=77.0 zc=65.0 d=23.0
2024-06-06 18:09:44.891 | INFO     | __main__:<module>:38 - 93-1.3.6.1.4.1.55648.2.736427593689433730094955065877066699507-1
2024-06-06 18:09:51.266 | DEBUG    | __main__:<module>:19 - xc=222.5 yc=2

2024-06-06 18:10:51.633 | DEBUG    | __main__:<module>:19 - xc=367.5 yc=248.0 w=66.0 h=71.0 zc=31.0 d=21.0
2024-06-06 18:10:52.738 | INFO     | __main__:<module>:38 - 103-1.3.6.1.4.1.55648.2.835474881974104132193144026122007786638-1
2024-06-06 18:10:59.563 | DEBUG    | __main__:<module>:19 - xc=359.0 yc=305.0 w=85.0 h=63.0 zc=74.5 d=14.0
2024-06-06 18:10:59.662 | DEBUG    | __main__:<module>:19 - xc=366.0 yc=342.0 w=11.0 h=9.0 zc=35.0 d=1.0
2024-06-06 18:10:59.759 | DEBUG    | __main__:<module>:19 - xc=342.0 yc=342.0 w=7.0 h=7.0 zc=34.0 d=1.0
2024-06-06 18:10:59.860 | DEBUG    | __main__:<module>:19 - xc=153.0 yc=143.0 w=7.0 h=7.0 zc=70.0 d=1.0
2024-06-06 18:10:59.959 | DEBUG    | __main__:<module>:19 - xc=132.5 yc=330.5 w=8.0 h=8.0 zc=38.0 d=1.0
2024-06-06 18:11:01.037 | INFO     | __main__:<module>:38 - 104-1.3.6.1.4.1.55648.2.856278606190857063618470110437468682301-5
2024-06-06 18:11:06.551 | DEBUG    | __main__:<module>:19 - xc=170.0 yc=113.5 w=59.0 h=228.0 zc=77.0 d=7.0
2024-06-06

2024-06-06 18:11:57.604 | DEBUG    | __main__:<module>:19 - xc=151.5 yc=261.0 w=12.0 h=7.0 zc=110.5 d=2.0
2024-06-06 18:11:58.562 | INFO     | __main__:<module>:38 - 114-1.3.6.1.4.1.55648.2.909562561349167665007734640640520520584-2
2024-06-06 18:12:03.709 | DEBUG    | __main__:<module>:19 - xc=212.0 yc=253.5 w=47.0 h=42.0 zc=46.0 d=13.0
2024-06-06 18:12:09.230 | INFO     | __main__:<module>:38 - 115-1.3.6.1.4.1.55648.2.913239413111188792632935164607668272947-1
2024-06-06 18:12:23.531 | DEBUG    | __main__:<module>:19 - xc=362.5 yc=183.5 w=68.0 h=68.0 zc=86.0 d=13.0
2024-06-06 18:12:24.999 | INFO     | __main__:<module>:38 - 116-1.3.6.1.4.1.55648.2.913637523551865938586481818051290402649-1
2024-06-06 18:12:31.196 | DEBUG    | __main__:<module>:19 - xc=142.0 yc=227.0 w=13.0 h=17.0 zc=113.0 d=1.0
2024-06-06 18:12:31.311 | DEBUG    | __main__:<module>:19 - xc=269.0 yc=200.5 w=221.0 h=46.0 zc=106.0 d=1.0
2024-06-06 18:12:31.429 | DEBUG    | __main__:<module>:19 - xc=140.5 yc=346.0 w=58.0 h=

2024-06-06 18:13:38.986 | DEBUG    | __main__:<module>:19 - xc=359.5 yc=270.5 w=96.0 h=88.0 zc=60.5 d=26.0
2024-06-06 18:13:39.932 | INFO     | __main__:<module>:38 - 127-1.3.6.1.4.1.55648.2.951349899544548320020499945618796321859-3
2024-06-06 18:13:43.731 | DEBUG    | __main__:<module>:19 - xc=189.5 yc=295.5 w=86.0 h=80.0 zc=68.5 d=20.0
2024-06-06 18:13:43.817 | DEBUG    | __main__:<module>:19 - xc=342.0 yc=199.5 w=11.0 h=8.0 zc=71.5 d=2.0
2024-06-06 18:13:43.898 | DEBUG    | __main__:<module>:19 - xc=427.5 yc=190.0 w=14.0 h=9.0 zc=45.5 d=2.0
2024-06-06 18:13:44.720 | INFO     | __main__:<module>:38 - 128-1.3.6.1.4.1.55648.2.958354693790071181336523988022568927193-3
2024-06-06 18:13:50.636 | DEBUG    | __main__:<module>:19 - xc=227.5 yc=327.0 w=48.0 h=43.0 zc=46.5 d=32.0
2024-06-06 18:13:51.646 | INFO     | __main__:<module>:38 - 129-1.3.6.1.4.1.55648.2.963888885476196307473826820010553815488-1
2024-06-06 18:13:58.505 | DEBUG    | __main__:<module>:19 - xc=332.5 yc=235.5 w=44.0 h=52.0

In [1]:
from safetensors.numpy import load_file



In [2]:
fl = load_file("/cache/fast_data_nas8/qct/shubham/cache_3d/wcg/1.3.6.1.4.1.55648.297188423527540138421280929506759090439.7.safetensors")

In [3]:
fl["images"].shape

(64, 242, 388)

In [7]:
fl["boxes"].shape

(1, 6)

In [5]:
import numpy as np
import torch
from safetensors.numpy import load_file

# Path to your saved safetensors file
file_path = 'your_saved_file.safetensors'

# Load the numpy arrays from the safetensors file
data =fl

# Convert numpy arrays to PyTorch tensors
tensors = {key: torch.tensor(value) for key, value in data.items()}

# Example usage
for key, tensor in tensors.items():
    print(f'{key}: {tensor.shape}')


boxes: torch.Size([1, 6])
images: torch.Size([64, 242, 388])
labels: torch.Size([1])
lung_box: torch.Size([6])
rads: torch.Size([1])
spacing: torch.Size([3])


In [10]:
def _bbox3d_f2in(bbox):
    # convert a 3d bounding from float to int
    bbox[:, :3] = np.floor(bbox[:, :3])
    bbox[:, 3:] = np.ceil(bbox[:, 3:])
    bbox = bbox.astype(np.int32)
    return bbox

def _bbox3d_f2in(bbox):
    # convert a 3d bounding box from float to int
    bbox[:, :3] = torch.floor(bbox[:, :3])
    bbox[:, 3:] = torch.ceil(bbox[:, 3:])
    bbox = bbox.int()
    return bbox

In [11]:
_bbox3d_f2in(tensors["boxes"])

tensor([[ 37, 142, 329,  39, 150, 340]], dtype=torch.int32)

In [6]:
tensors

{'boxes': tensor([[ 37.5000, 142.5000, 329.5000,  38.5000, 149.5000, 339.5000]],
        dtype=torch.float64),
 'images': tensor([[[ -984,  -990,  -999,  ..., -1002,  -997,  -989],
          [ -995,  -997,  -998,  ...,  -997,  -990,  -983],
          [ -994,  -997,  -993,  ...,  -994,  -995, -1000],
          ...,
          [ -932,  -941,  -916,  ...,  -808,  -850,  -884],
          [ -920,  -933,  -929,  ...,  -852,  -888,  -896],
          [ -921,  -925,  -930,  ...,  -883,  -892,  -892]],
 
         [[ -995, -1002,  -995,  ..., -1030, -1014, -1006],
          [ -994,  -996,  -993,  ..., -1014, -1004,  -998],
          [ -989,  -993,  -987,  ..., -1002, -1005, -1006],
          ...,
          [ -891,  -867,  -837,  ...,  -658,  -726,  -794],
          [ -927,  -905,  -889,  ...,  -749,  -821,  -871],
          [ -926,  -921,  -913,  ...,  -835,  -892,  -899]],
 
         [[-1005,  -991,  -988,  ..., -1007, -1006, -1002],
          [-1004,  -996,  -988,  ..., -1006, -1008, -1004],
   

In [ ]:
import numpy as np
import torch
from safetensors import safe_open

# Path to your saved safetensors file
file_path = 'your_saved_file.safetensors'

# Function to load numpy data from safetensors file and convert to tensor
def load_safetensors_as_tensor(file_path):
    with safe_open(file_path, framework="numpy") as f:
        data = {k: np.array(v) for k, v in f.items()}
    
    # Convert numpy arrays to tensors
    tensors = {k: torch.tensor(v) for k, v in data.items()}
    return tensors

# Load and convert
tensors = load_safetensors_as_tensor(file_path)

# Example usage
for key, tensor in tensors.items():
    print(f'{key}: {tensor.shape}')


In [7]:
from qct_data.series_meta import get_qct_collection
from qct_utils.schema.ct.common import CTScan
scans_meta = get_qct_collection("recist_segmed")
sid = "1.3.6.1.4.1.55648.2.145415111583706588050030447094741143985"
series_dict = scans_meta.find_one({"_id": sid})


ctimg = CTScan.load(
    series_dict, readtype="dcm", scan_type="chestct", gaussian_smoothing_sigma=None
)

In [8]:
from qct_utils.ctvis.viewer import plot_scans

In [10]:
plot_scans([ctimg.array] , [""])

interactive(children=(RadioButtons(description='Slice plane selection:', index=2, options=('y-z', 'z-x', 'x-y'…

In [29]:
#| hide
import nbdev; nbdev.nbdev_export()